In [8]:
import time
import os
import seaborn as sns
import numpy as np
import pandas as pd
import sys
%load_ext autoreload
%autoreload 2
from mimas.tools.spectral_file.extract_ms1_feature import process_mzml_file, extract_ms2_spectra, find_features
import logging
from toolsets.spectra_operations import entropy_similarity_default
from toolsets.std_list_prep import complete_adducts, complete_formula
# from toolsets.API_gets import complete_smiles
from toolsets.search import string_search, num_search
import toolsets.spectra_operations as so
from tqdm import tqdm
from toolsets.features_by_alphapept import find_features_alphapept
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from toolsets.file_io import parse_file_name
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.Descriptors import ExactMolWt
from toolsets.file_io import parse_file_name
from toolsets.std_list_prep import all_adduct_neg, all_adduct_pos, calculate_precursormz
source_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/core_data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def guess_method(data):
    methods=data.iloc[0]['method'].split('|')
    if methods[-1].strip() =='negative':
        return(-1)
    elif methods[-1].strip() =='positive':
        return(1)
    else:
        return(0)
def guess_adduct(input):
    from toolsets.std_list_prep import all_adduct_neg, all_adduct_pos
    adduct_guess = []
    calculated_precursor =[]
    mz_offset = []

    mode = guess_method(input)
    if mode ==-1:
        all_adduct = all_adduct_neg
    elif mode ==1:
        all_adduct = all_adduct_pos
    else:
        all_adduct = []
    for index, row in input.iterrows():
        premz_list = []
        for adduct in all_adduct:
            try:
                premz_list.append(calculate_precursormz(row['smiles'], adduct))
            except:
                break
        if len(premz_list)>0:
            premz_diff = ([abs(x - float(row['precursor_mass'])) for x in premz_list])
            index_min = np.argmin(premz_diff)
            if premz_diff[index_min]<0.01:
                adduct_guess.append(all_adduct[index_min])
                calculated_precursor.append(premz_list[index_min])
                mz_offset.append(premz_list[index_min]-float(row['precursor_mass']))
            else:
                adduct_guess.append('no_valid_adduct_found')
                calculated_precursor.append(np.NAN)
                mz_offset.append(np.NAN)
        else:
             adduct_guess.append('smile_parsing_error')
             mz_offset.append(np.NAN)
             calculated_precursor.append(np.NAN)
    input['adduct_guess']=adduct_guess
    input['calculated_precursor']=calculated_precursor
    input['mz_offset']=mz_offset
    return(input)

In [2]:
# source_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/core_data'
# filename = '5m_hilic_orbitrap_beh_amide_negative_12-2022 for core mzrt list.csv'
#
# files= find_files(source_dir, '.csv')
# files

In [11]:
file_name =  '5m_splash_one_orbitrap_beh_c18_positive_12-08-2022 for core mz rt list_with int stds.csv'
input = pd.read_csv(os.path.join(source_dir, '5m_splash_one_orbitrap_beh_c18_positive_12-08-2022 for core mz rt list_with int stds.csv'))
input.dropna(how='all', inplace=True,axis=1)

In [10]:
input

,splash,name,retention_index,precursor_mass,pre_cursors_intensity,adduct,method,inchi_key,PubChem CID,smiles
0,splash10-00kr-9116000000-314987c24cb5ab7397c9,1_CAR 10:0-d3 iSTD [M]+,13,319.268,NaN,NaN,5m splash one | orbitrap | beh c18 | positive,NaN,NaN,NaN
1,splash10-000i-9013000000-c5bb3d70211dc8040d65,1_CAR 12:0-d3 iSTD [M]+,17,347.299,NaN,NaN,5m splash one | orbitrap | beh c18 | positive,NaN,NaN,NaN
2,splash10-000i-9001300000-ab75b83b8573f2895d96,1_CAR 18:0-d3 iSTD [M]+,66,431.393,NaN,NaN,5m splash one | orbitrap | beh c18 | positive,NaN,NaN,NaN
3,splash10-0a59-9848004000-8ba348ffcbd381238970,1_CE 18:1-d7 iSTD [M+Na]+,258,680.633,NaN,NaN,5m splash one | orbitrap | beh c18 | positive,NaN,NaN,NaN
4,splash10-004i-0103000900-e702fa7ca9a7f684a43e,1_CE 22:1 iSTD [M+Na]+,270,729.652,NaN,NaN,5m splash one | orbitrap | beh c18 | positive,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
981,splash10-0079-1900000000-25d7c9d9b8bdc820e06e,urocanic acid,271,139.050,2000000.0,M+H,5m splash one | orbitrap | beh c18 | positive,LOIYMIARKYCTBW-OWOJBTEDSA-N,736715.0,C1=C(NC=N1)/C=C/C(=O)O
982,splash10-03di-0900000000-2af1e8792fceeb2eab68,valerophenone,170,163.112,10000000.0,M+H,5m splash one | orbitrap | beh c18 | positive,XKGLSKVNOSHTAD-UHFFFAOYSA-N,66093.0,CCCCC(=O)C1=CC=CC=C1
983,splash10-004i-6890000000-9ceecadc5c1c9b711c9d,vitamin K1,79,226.181,2000000.0,NaN,5m splash one | orbitrap | beh c18 | positive,MBWXNTAXLNYFJB-UHFFFAOYSA-N,4812.0,CC(=CCC1=C(C)C(=O)c2ccccc2C1=O)CCCC(C)CCCC(C)C...
984,splash10-001i-3900000000-d4dd4121bd9e6cb8e4c2,w-hydroxydodecanoic acid,78,181.159,500000.0,NaN,5m splash one | orbitrap | beh c18 | positive,ZDHCZVWCTKTBRY-UHFFFAOYSA-N,79034.0,C(CCCCCC(=O)O)CCCCCO


In [12]:
input_return = guess_adduct(input)
input_return.to_csv(os.path.join(source_dir,file_name.split('.')[0]+'_completed'+'.csv'))

In [77]:
# from toolsets.file_io import find_files
# files= find_files(source_dir, '.csv')
# for file in files:
#     input = pd.read_csv(os.path.join(source_dir, file))
#     input.dropna(how='all', inplace=True,axis=1)
#     input_return = guess_adduct(input)
#     input_return.to_csv(os.path.join(source_dir,file.split('.')[0]+'_completed'+'.csv'))

[14:13:43] SMILES Parse Error: syntax error while parsing: na
[14:13:43] SMILES Parse Error: Failed parsing SMILES 'na' for input: 'na'
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors
[14:13:43] WARNING: not removing hydrogen atom without neighbors


In [4]:
input = pd.read_csv(os.path.join(source_dir, filename))